In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [3]:
from pyspark.sql.types import (StructType, StructField, IntegerType, 
                               DoubleType, DateType)

schema = StructType([StructField('ID', IntegerType(), True),
                     StructField('Date', DateType(), True),
                     StructField('Open', DoubleType(), True),
                     StructField('High', DoubleType(), True),
                     StructField('Low', DoubleType(), True),
                     StructField('Close', DoubleType(), True),
                     StructField('Adj Close', DoubleType(), True),
                     StructField('Volume', IntegerType(), True)
                    ])

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [4]:
df = spark.readStream.format('csv')\
.schema(schema)\
.load('InputStream/InputStream/')

### Make sure the dataframe is streaming the files from the folder

In [5]:
df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [6]:
writer = df.writeStream.outputMode('append')\
.format('memory')\
.queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [7]:
query = writer.start()

23/10/06 14:07:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8c924f63-0235-4e50-9bb9-7f05d4c3856d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/10/06 14:07:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
df_mem = spark.sql('SELECT * FROM stock')
df_mem.show(50)

+---+----+----+----+---+-----+---------+------+
| ID|Date|Open|High|Low|Close|Adj Close|Volume|
+---+----+----+----+---+-----+---------+------+
+---+----+----+----+---+-----+---------+------+



### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [9]:
df_null_out = df.na.drop(how='all') 

23/10/06 14:07:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [11]:
df_modified = df_null_out.withColumn('diff', col('High') - col('Low'))

In [12]:
query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

### Create a new write stream using the new generated dataframe and call the generated table "modified_data"

In [13]:
new_writer = df_modified.writeStream.outputMode('append')\
.format('memory')\
.queryName('modified_data')

In [14]:
query = new_writer.start()

23/10/06 14:07:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-41128214-c2f2-41a7-8744-859e0bc0f47d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/10/06 14:07:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [15]:
finalDf = spark.sql('''select *
                       from  modified_data
                        ''')
finalDf.show(30)

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|1421.5996099999975|
|121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651| 995.0996099999975|
|122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209| 924.0996090000008|
|123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|1990.3007819999984|
|124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|1599.4003909999992|
|125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|1777.0996090000008|
|126|2000-06-28|23884.199219

### Write the generated data into files instead of the memory. 

In [16]:
file_writer = df_modified.writeStream.outputMode('append')\
        .format('csv')\
        .option('path', 'Outstream/')\
        .option('checkpointLocation', 'chkpnt')

In [17]:
query = file_writer.start()

23/10/06 14:09:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


### Stop the query. Now, try reading the generated files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [22]:
query.stop()

In [18]:
finalschema = StructType([StructField('ID', IntegerType(), True),
                         StructField('Date', DateType(), True),
                         StructField('Open', DoubleType(), True),
                         StructField('High', DoubleType(), True),
                         StructField('Low', DoubleType(), True),
                         StructField('Close', DoubleType(), True),
                         StructField('Adj Close', DoubleType(), True),
                         StructField('Volume', IntegerType(), True),
                         StructField('diff', DoubleType(), True)
                    ])

In [19]:
lastdf = spark.read.format('csv')\
.schema(finalschema)\
.load('Outstream')

In [20]:
lastdf.show(30)

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|120|2000-06-20|22817.900391|23102.199219|21680.599609|22320.300781|21092.632813| 34466|1421.5996099999975|
|121|2000-06-21|21893.800781|22675.699219|21680.599609|22675.699219|21428.484375| 68651| 995.0996099999975|
|122|2000-06-22|23386.599609|23386.599609|     22462.5|23031.099609|21764.335938| 97209| 924.0996090000008|
|123|2000-06-23|22107.099609|24097.400391|22107.099609|     22889.0|21630.052734|199483|1990.3007819999984|
|124|2000-06-26|23102.199219|     24168.5|22569.099609|24026.300781|22704.796875|121969|1599.4003909999992|
|125|2000-06-27|24026.300781|25519.099609|     23742.0|24026.300781|22704.796875|113809|1777.0996090000008|
|126|2000-06-28|23884.199219

### Sort the dataframe based on the ID

In [21]:
lastdfSorted = lastdf.sort('ID')
lastdfSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|2878.9003900000025|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|            2559.0|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746| 1919.298827999999|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|2843.3007810000017|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|            1706.0|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|1066.3007810000017|
|  6|2000-01-12|     24168.5